# Obtaining the growth data from the Forest Vegetation Simulator that will serve as reference for the calibration

The [Forest Vegetation Simulator](https://www.fs.usda.gov/fvs/) (FVS) model was develloped by the US department of Agriculture and Forestry (USDA). It aims to predict the growth and yield of forest stands based on internal equations derived for empirical data, but it also have options to be calibrated or adjusted based on the input data.

Eric Gustafson recommands its use for the first step of the calibration procedure of PnET Succession (see [Gustafson and Miranda (2023)](./ReferencesAndData/Gustafson2024PnETUserGuide.pdf)), where we try to make sure that the growth curve of a new cohort matches what is seen in reality. This is because the curves produced by FVS can be easily compared to PnET, without the trouble that often comes when using real empirical data for such a procedure (as empirical data is messy, needs to be cleaned, has outliers, etc.). As such, using FVS to calibrate PnET is called by some "meta-modelling" - as empirical data has been used to create the FVS model, which is then used to calibrate PnET.

The FVS model is already compiled inside of the Docker image used to produced this document (see the [readme file](https://github.com/Klemet/Calibration_PnET_DIVERSE/blob/main/README.md) to deploy it easily on your computer), thanks to the code provided in [this issue](https://github.com/USDAForestService/ForestVegetationSimulator/issues/44#issuecomment-2483578729). What that means is that FVS can be run from Python functions that I created here (see [functionsForCalibration.py](./functionsForCalibration.py)) without any need for manually creating the intputs for a FVS simulation (or projection, as it is more usually called in the user guides); or to manually analyze the outputs of FVS.

## Custom python function used to obtain growth data from FVS for a monoculture

The function `FVS_on_simulationOnSingleEmptyStand` (inside [functionsForCalibration.py](./functionsForCalibration.py)) will be main one that we will be using to calibrate PnET.

This function does the following :

- It creates a MySQL database containg the stand and trees inputs necessary for a simple FVS simulation that contains only young trees of a single species (monoculture) in ideal or default conditions at the beginning.
    - These initial conditions are used to replicate what happens when we initialize a new age cohort on an empty cell in PnET, in order to do the calibration on monoculture and ideal conditions.
    - While the function allows the user to specify the number of trees, I will use 2471 trees (which will initialize a 1ha stand with 2471 trees) which is the value used in [Reese et al. 2024, supp. mat. A](https://cdnsciencepub.com/doi/10.1139/cjfr-2024-0085) (recommanded by [Brian Sturtevant](https://research.fs.usda.gov/about/people/sturtevant)).
    - The function also allows the user to specify the "Site index" for the stand, which is a value representing the quality of the soil for the species (and is thus used to increase tree growth in the equations inside FVS). [Reese et al. 2024, supp. mat. A](https://cdnsciencepub.com/doi/10.1139/cjfr-2024-0085) tend to use high site index as the reference to calibrate PnET, which matches the advice of [Gustafson and Miranda (2023)](./ReferencesAndData/Gustafson2024PnETUserGuide.pdf) (who recommands that this first calibration step should be done on "ideal conditions"). As the Site Index is a value that varies from species to species, I recommand [this report from the USDA](https://www.fs.usda.gov/nrs/pubs/gtr/gtr_nc128.pdf) that shows ranges of values of Site Index in the USA for many species. I will also make several FVS simulations with lower site index values for comparison.
- I creates a simple keyword file, which is the file that defines what stands are simulated, and how.
    - The keyword file generated simply simulate one stand with a single tree age cohort, and output carbon estimates in the main output file (which are then used to derive aboveground biomass estimates we can compare to PnET)
- It runs the simulation by calling the right FVS "variant" as indicated by the user
    - There exists more than a dozen FVS variants for different regions of the united states, but also one for Ontario and one for British Columbia. I'll be using the Ontario variante here by default.
- It reads the outputs to return a dictionnary of values that associates a year (e.g. 2033) to an aboveground biomass value. Convertion from carbon to biomass is explained in the code of the function.
- The files generated for (inputs) and by (outputs) the FVS simulation are deleted by default (`clearFiles` argument), but this can be overriden.

## Example

In [1]:
# We import the functions
from functionsForCalibration import *

# We run FVS three times to simulate a basalm fir monoculture on an initially empty stand, and specify all of the arguments

basalmFir_LowSiteIndex = FVS_on_simulationOnSingleEmptyStand(Latitude = 47.22029, # Atikamekw village of Manawan, center of one of our study areas
                                    Longitude = -74.38606, # Atikamekw village of Manawan, center of one of our study areas
                                    Slope = 0, # We want ideal/neutral conditions, so no slope
                                    Elevation = 0, # We want ideal/neutral conditions, so no elevation 
                                    treeSpeciesCode = "BF", # Balsam fir. All codes are available at https://github.com/USDAForestService/ForestVegetationSimulator/blob/5c29887e4168fd8182c1b2bad762f900b7d7e90c/canada/on/grinit.f#L43
                                    numberOfTrees = 2471, # To replicate stocking density from  https://cdnsciencepub.com/doi/10.1139/cjfr-2024-0085, supp. mat. A
                                    siteIndex = 20, # Low site index for Balsam fir. See https://www.fs.usda.gov/nrs/pubs/gtr/gtr_nc128.pdf
                                    variant = "FVSon",
                                    folderForFiles = "/calibrationFolder/FVS_SingleEmptyStandRun",
                                    clearFiles = True)

basalmFir_averageSiteIndex = FVS_on_simulationOnSingleEmptyStand(Latitude = 47.22029, # Atikamekw village of Manawan, center of one of our study areas
                                    Longitude = -74.38606, # Atikamekw village of Manawan, center of one of our study areas
                                    Slope = 0, # We want ideal/neutral conditions, so no slope
                                    Elevation = 0, # We want ideal/neutral conditions, so no elevation 
                                    treeSpeciesCode = "BF", # Balsam fir. All codes are available at https://github.com/USDAForestService/ForestVegetationSimulator/blob/5c29887e4168fd8182c1b2bad762f900b7d7e90c/canada/on/grinit.f#L43
                                    numberOfTrees = 2471, # To replicate stocking density from  https://cdnsciencepub.com/doi/10.1139/cjfr-2024-0085, supp. mat. A
                                    siteIndex = 45, # Average site index for Balsam fir. See https://www.fs.usda.gov/nrs/pubs/gtr/gtr_nc128.pdf
                                    variant = "FVSon",
                                    folderForFiles = "/calibrationFolder/FVS_SingleEmptyStandRun",
                                    clearFiles = True)

basalmFir_HighSiteIndex = FVS_on_simulationOnSingleEmptyStand(Latitude = 47.22029, # Atikamekw village of Manawan, center of one of our study areas
                                    Longitude = -74.38606, # Atikamekw village of Manawan, center of one of our study areas
                                    Slope = 0, # We want ideal/neutral conditions, so no slope
                                    Elevation = 0, # We want ideal/neutral conditions, so no elevation 
                                    treeSpeciesCode = "BF", # Balsam fir. All codes are available at https://github.com/USDAForestService/ForestVegetationSimulator/blob/5c29887e4168fd8182c1b2bad762f900b7d7e90c/canada/on/grinit.f#L43
                                    numberOfTrees = 2471, # To replicate stocking density from  https://cdnsciencepub.com/doi/10.1139/cjfr-2024-0085, supp. mat. A
                                    siteIndex = 70, # High site index for Balsam fir. See https://www.fs.usda.gov/nrs/pubs/gtr/gtr_nc128.pdf
                                    variant = "FVSon",
                                    folderForFiles = "/calibrationFolder/FVS_SingleEmptyStandRun",
                                    clearFiles = True)

The directory '/calibrationFolder/FVS_SingleEmptyStandRun' has been created.
Creating Database with stand and tree ini values
Creating Keyword file
Launching FVS sim

                  ON FVS METRIC VARIANT -- RV:20221118  


        STAND = STAND_EMPTY                 MANAGEMENT CODE = NONE


               SUMMARY STATISTICS (BASED ON TOTAL STAND AREA)
----------------------------------------------------------------------------
      START OF SIMULATION PERIOD    REMOVALS/HA     AFTER TREATMENT   GROWTH
     ---------------------------- ----------------- ----------------    CU M
     TREES         TOP        GTV TREES   GTV   GMV         TOP       PER YR
YEAR   /HA  BA SDI  HT  QMD  CU M   /HA  CU M  CU M  BA SDI  HT  QMD ACC MOR
---- ----- --- --- --- ---- ----- ----- ----- ----- --- --- --- ---- --- ---
2023  2471   0   5   2  0.5     0     0     0     0   0   5   2  0.5   0   0
2033  2402   2  74   3  2.9     0     0     0     0   2  74   3  2.9   0   0
2043  2335   8 284   6  6.8


                  ON FVS METRIC VARIANT -- RV:20221118  


        STAND = STAND_EMPTY                 MANAGEMENT CODE = NONE


               SUMMARY STATISTICS (BASED ON TOTAL STAND AREA)
----------------------------------------------------------------------------
      START OF SIMULATION PERIOD    REMOVALS/HA     AFTER TREATMENT   GROWTH
     ---------------------------- ----------------- ----------------    CU M
     TREES         TOP        GTV TREES   GTV   GMV         TOP       PER YR
YEAR   /HA  BA SDI  HT  QMD  CU M   /HA  CU M  CU M  BA SDI  HT  QMD ACC MOR
---- ----- --- --- --- ---- ----- ----- ----- ----- --- --- --- ---- --- ---
2023  2471   0   5   2  0.5     0     0     0     0   0   5   2  0.5   0   0
2033  2402   2  74   3  2.9     0     0     0     0   2  74   3  2.9   0   0
2043  2335   8 284   6  6.8     0     0     0     0   8 284   6  6.8   0   0
2053  1871  19 511   9 11.3     0     0     0     0  19 511   9 11.3  12   0
2063  1670  25 620  11 13.7   118     0

In [2]:
# We print the outputs of one of the simulations
print(basalmFir_HighSiteIndex)

{2023: 0.4, 2033: 2.6, 2043: 21.6, 2053: 61.0, 2063: 86.4, 2073: 113.6, 2083: 134.4, 2093: 155.6, 2103: 175.8, 2113: 196.6, 2123: 216.8, 2133: 239.6}


```{admonition} Other uses of FVS
:class: tip
FVS can also output belowground biomass (both live and dead) using the CARBPRET (carbon report) keyword.
This can be useful to paramterize several parameters of PnET Succession related to the biomass of roots.
```

```{admonition} Learning more about FVS
:class: note
FVS is a complex model with more than a decade of improvments. The existing Python functions present here should be enough to calibrate PnET Succession; but if you need more information about variants, possible outputs and inputs, you can take a look at the many [user guides](https://www.fs.usda.gov/fvs/documents/guides.shtml) that are available online.
```

